In [7]:
#####training q_resnet18
import sys
import time
import random
import os
sys.path.append("..")
from module.layer_fusion_module import *
from module.resnet_module import *
from module.extract_weight_module import *
model_dir = "saved_models"
model_filename = "q_resnet18_cifar10.pt"
fusioned_model_filename = "q_fusioned_resnet18_cifar10.pt"
quantized_model_filename_jit = "q_resnet18_quantized_cifar10.pt"
quantized_model_filename = "q_resnet18_quantized_cifar10.pt"
model_filepath = os.path.join(model_dir, model_filename)
quantized_model_filepath = os.path.join(model_dir, quantized_model_filename)
quantized_model_filepath_jit = os.path.join(model_dir, quantized_model_filename_jit)

def fuse_model(model,model_name):
    fusion_layer_dict= make_fuse_dict(model,model_name)
    for key in fusion_layer_dict:
        for fuse_group in fusion_layer_dict[key]:        
            torch.quantization.fuse_modules(eval(key), [fuse_group], inplace=True) 

num_classes = 10
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

model = create_model(num_classes=num_classes)
model.to(cpu_device)

quantized_model = QuantizedResNet18(model_fp32=model)

quantization_config = torch.quantization.get_default_qconfig("fbgemm")

quantized_model.qconfig = quantization_config
fuse_model(quantized_model,'quantized_model')
torch.quantization.prepare_qat(quantized_model, inplace=True)
quantized_model.to(cpu_device)    
quantized_model = torch.quantization.convert(quantized_model, inplace=True)
#save_torchscript_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename_jit)
#save_model(model=quantized_model, model_dir=model_dir, model_filename=quantized_model_filename)
loaded_model=load_model(model=quantized_model, model_filepath=quantized_model_filepath, device='cpu')
loaded_model.eval()
#save each layer weights


QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0374]), zero_point=tensor([57]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (model_fp32): ResNet(
    (conv1): Sequential(
      (0): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.08831730484962463, zero_point=0, padding=(3, 3))
      (1): Identity()
      (2): Identity()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (conv2_x): Sequential(
      (0): BasicBlock(
        (residual_function): Sequential(
          (0): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.031286418437957764, zero_point=0, padding=(1, 1))
          (1): Identity()
          (2): Identity()
          (3): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.07724595814943314, zero_point=68, padding=(1, 1))
          (4): Identity()
        )
        (shortcut): Sequential()
        (relu): ReLU()
      )
      (1): BasicBlock(
        (residual

In [8]:
extract_model=loaded_model
save_list= dict()
save_list=make_default_save_file(save_list)
for parameter_name in extract_model.state_dict():
    print(parameter_name)
    if parameter_name[-6:]=='weight':
        file_name=parameter_name
        save_np_file=extract_model.state_dict()[parameter_name].int_repr().numpy().flatten()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        file_name=parameter_name+"_per_channel_scales"
        save_np_file=extract_model.state_dict()[parameter_name].q_per_channel_scales().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        file_name=parameter_name+"_per_channel_zero_points"
        save_np_file=extract_model.state_dict()[parameter_name].q_per_channel_zero_points().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
    elif parameter_name[-4:]=='bias':
        file_name=parameter_name
        save_np_file=extract_model.state_dict()[parameter_name].detach().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        
    elif parameter_name[-14:]=='_packed_params':
        file_name=parameter_name
        save_np_file=extract_model.state_dict()[parameter_name][0].int_repr().numpy().flatten()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        file_name=parameter_name+"_per_channel_scales"
        save_np_file=extract_model.state_dict()[parameter_name][0].q_per_channel_scales().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        file_name=parameter_name+"_per_channel_zero_points"
        save_np_file=extract_model.state_dict()[parameter_name][0].q_per_channel_zero_points().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)
        file_name=parameter_name+"_bias"
        save_np_file=extract_model.state_dict()[parameter_name][1].detach().numpy()
        save_list=save_np_1darray(save_list,file_name,save_np_file)            
    else:
        print(extract_model.state_dict()[parameter_name])
    #else:
    #    print(model_int8_unfused.state_dict()[parameter_name].numpy())


quant.scale
tensor([0.0374])
quant.zero_point
tensor([57])
model_fp32.conv1.0.weight
model_fp32.conv1.0.bias
model_fp32.conv1.0.scale
tensor(0.0883)
model_fp32.conv1.0.zero_point
tensor(0)
model_fp32.conv2_x.0.residual_function.0.weight
model_fp32.conv2_x.0.residual_function.0.bias
model_fp32.conv2_x.0.residual_function.0.scale
tensor(0.0313)
model_fp32.conv2_x.0.residual_function.0.zero_point
tensor(0)
model_fp32.conv2_x.0.residual_function.3.weight
model_fp32.conv2_x.0.residual_function.3.bias
model_fp32.conv2_x.0.residual_function.3.scale
tensor(0.0772)
model_fp32.conv2_x.0.residual_function.3.zero_point
tensor(68)
model_fp32.conv2_x.1.residual_function.0.weight
model_fp32.conv2_x.1.residual_function.0.bias
model_fp32.conv2_x.1.residual_function.0.scale
tensor(0.0376)
model_fp32.conv2_x.1.residual_function.0.zero_point
tensor(0)
model_fp32.conv2_x.1.residual_function.3.weight
model_fp32.conv2_x.1.residual_function.3.bias
model_fp32.conv2_x.1.residual_function.3.scale
tensor(0.1420)


In [9]:
#saved layer info
for name in save_list:
    print(name, "  data size:",save_list[name], "byte")
#merge weights
merge_weight_files(save_list)

default_setting   data size: 16 byte
model_fp32.conv1.0.weight   data size: 9408 byte
model_fp32.conv1.0.weight_per_channel_scales   data size: 256 byte
model_fp32.conv1.0.weight_per_channel_zero_points   data size: 256 byte
model_fp32.conv1.0.bias   data size: 256 byte
model_fp32.conv2_x.0.residual_function.0.weight   data size: 36864 byte
model_fp32.conv2_x.0.residual_function.0.weight_per_channel_scales   data size: 256 byte
model_fp32.conv2_x.0.residual_function.0.weight_per_channel_zero_points   data size: 256 byte
model_fp32.conv2_x.0.residual_function.0.bias   data size: 256 byte
model_fp32.conv2_x.0.residual_function.3.weight   data size: 36864 byte
model_fp32.conv2_x.0.residual_function.3.weight_per_channel_scales   data size: 256 byte
model_fp32.conv2_x.0.residual_function.3.weight_per_channel_zero_points   data size: 256 byte
model_fp32.conv2_x.0.residual_function.3.bias   data size: 256 byte
model_fp32.conv2_x.1.residual_function.0.weight   data size: 36864 byte
model_fp32.